In [ ]:
from astropy import units as u
from astropy import nddata
from astropy.io import fits
from astropy.wcs import WCS
from astropy import visualization
from astropy.utils import data

In [ ]:
import os
import ipywidgets

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
datafn = data.download_file('http://www.stsci.edu/~etollerud/jclj01tvq_flc.fits.gz', cache=False)

If the link no longer works, you can try to get the file from MAST by searching on for Proposal ID 13745

In [ ]:
with fits.open(datafn) as ff:
    print(list(enumerate([hdu.name for hdu in ff])))
    wcs = WCS(ff[4], ff)
    d = nddata.CCDData(data=ff[4].data, uncertainty=ff[5].data, flags=ff[6].data, unit=u.count, wcs=wcs)

In [ ]:
ax = plt.subplot(projection=wcs)

plt.imshow(d.data, 
           norm=visualization.ImageNormalize(d.data, 
                                             interval=visualization.PercentileInterval(95), 
                                             stretch=visualization.AsinhStretch()))
ax.coords[1].set_ticks(spacing = 20*u.arcsec)
ax.coords[1].set_major_formatter('dd:mm:ss.s')

# Baseline test 

In [ ]:
import ginga
from ginga.web.jupyterw.ImageViewJpw import EnhancedCanvasView

In [ ]:
import requests
image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/e/e9/Felis_silvestris_silvestris_small_gradual_decrease_of_quality.png/240px-Felis_silvestris_silvestris_small_gradual_decrease_of_quality.png"
image = requests.get(image_url).content
img = ipywidgets.Image(format='jpeg', width=500, height=500, value=image)
img

In [ ]:
v1 = EnhancedCanvasView()
v1.set_widget(img)
v1.get_bindings().enable_all(True)

v1.set_cmap(ginga.cmap.get_cmap('viridis'))

v1.load_data(d.data)

In [ ]:
data_btn = ipywidgets.Button(description='data')
unc_btn = ipywidgets.Button(description='unc')
flag_btn = ipywidgets.Button(description='flags')
autocut = ipywidgets.Checkbox(description='autocut')
text = ipywidgets.Text('No cursor')

def update(newdata):
    oldpan = v1.get_pan()
    oldzoom = v1.get_zoom()
    
    v1.load_data(newdata)
    if autocut.value:
        v1.auto_levels()
    
    v1.set_pan(*oldpan)
    v1.zoom_to(oldzoom)

def data_click(button):
    update(d.data)
def unc_click(button):
    update(d.uncertainty.array)
def flag_click(button):
    update(d.flags)
    
data_btn.on_click(data_click)
unc_btn.on_click(unc_click)
flag_btn.on_click(flag_click)

hbox = ipywidgets.HBox([data_btn, unc_btn, flag_btn, autocut, text])
vbox = ipywidgets.VBox([img, hbox])
vbox

In [ ]:
#ff = fits.open('/Users/erik/projects/piscdwarfs_hst/raw/jclj01tvq_flc.fits')
#v1.get_image().wcs.load_header(ff[4].header, ff)

In [ ]:
v1.cut_levels(-4, 500)

In [ ]:
v1.set_pan(688,1141)

In [ ]:
v1.zoom_to(2)

In [ ]:
v1.markl(600, 700, 'circle')

* Build a class to do the above and put it in ginga or st_ginga if Eric is not interested
* tvmark and drawing plugin is a good example of marking (add layer and then draw into *that*)
* user can set shape programatically but click to get the mark

In [ ]:
arg = []

def f(*args, **kwargs):
    arg.append((args, kwargs))
    data_x, data_y = args[2:4]
    ra, dec = wcs.all_pix2world(data_x, data_y, 0)
    text.value = '{:.4},{:.4}'.format(float(ra), float(dec))

v1.add_callback('cursor-changed', f)

# Fixes to suggest

* `EnhancedCanvasView.get_color_algorithm` should exist

# Experiments with widgets 

In [ ]:
text =['first', 'second']
button = ipywidgets.Button(description="Click Me!")
button.i = 0

def on_button_clicked(b):
    b.description = text[b.i % len(text)] + str(b.i)
    b.i += 1

button.on_click(on_button_clicked)
button

In [ ]:
text.append('third')
on_button_clicked(button)

# Test below 

In [ ]:
%%javascript
/* some magic to keep the cell contents from scrolling
 */
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# create the beginnings of an image viewing application
import requests
image_url = "http://madamsabi.com/image/cache/data/catfish-500x500.jpg"
image = requests.get(image_url).content
jup_img = ipywidgets.Image(format='jpeg', width=500, height=500, value=image)

zoom = ipywidgets.FloatSlider(min=-20, max=20, step=1, continuous_update=True,
                          orientation='horizontal')
ipywidgets.VBox([jup_img, zoom])

In [ ]:
# Connect a Ginga viewer to this image widget
from ginga.misc.log import get_logger
logger = get_logger("my viewer", log_stderr=False, 
                    log_file=os.path.join(os.environ['HOME'], 'tmp' ,'ginga.log'), 
                    level=40)

from ginga.web.jupyterw.ImageViewJpw import EnhancedCanvasView
v1 = EnhancedCanvasView(logger=logger)
# set our linkage between the jupyter widget at ginga
v1.set_widget(jup_img)

# enable all possible keyboard and pointer operations
bd = v1.get_bindings()
bd.enable_all(True)

In [ ]:
from ginga.misc.log import get_logger

In [ ]:
v1.get_pan()

In [ ]:
Viewer('hgjghfjh')